In [1]:
import os
import sys
print(sys.path)
import BioSimSpace as BSS

['/opt/miniforge/envs/openbiosim/lib/python39.zip', '/opt/miniforge/envs/openbiosim/lib/python3.9', '/opt/miniforge/envs/openbiosim/lib/python3.9/lib-dynload', '', '/home/purohit.52@pharmacy.ohio-state.edu/.local/lib/python3.9/site-packages', '/opt/miniforge/envs/openbiosim/lib/python3.9/site-packages']


INFO:rdkit:Enabling RDKit 2023.09.6 jupyter extensions
INFO:numexpr.utils:Note: NumExpr detected 16 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.
/opt/miniforge/envs/openbiosim/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
/opt/miniforge/envs/openbiosim/lib/python3.9/pty.py:85: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [2]:
main_folder = os.getcwd()
input_dir = f"{main_folder}/inputs"
path_to_ligands = f"{main_folder}/inputs/ligands"
path_to_protein = f"{main_folder}/inputs"
# make a directory for all our output files
if not os.path.exists(f"{main_folder}/output_setup"):
    os.mkdir(f"{main_folder}/output_setup")

In [3]:
atp1 = BSS.IO.readMolecules(["inputs/ligands/2qn6-gtp.mol2"])[0] 
atp1 = BSS.Parameters.gaff(atp1).getMolecule()
BSS.IO.saveMolecules("atp1", atp1, ["gro87", "grotop"])
atp2 = BSS.IO.readMolecules(["inputs/ligands/gtp_maegher.prm7","inputs/ligands/gtp_maegher.rst7"])[0]   
BSS.IO.saveMolecules("atp2", atp2, ["gro87", "grotop"])

['/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.gro',
 '/home/purohit.52@pharmacy.ohio-state.edu/Desktop/RBFE/bss_fep_advanced_tut_bg-main/ATP-ADP_RBFE/atp2.top']

In [4]:
mapping = BSS.Align.matchAtoms(atp1, atp2)
print(mapping)

{0: 5, 1: 7, 2: 6, 3: 4, 4: 8, 5: 9, 6: 11, 7: 10, 8: 12, 9: 13, 10: 16, 11: 18, 12: 36, 13: 37, 14: 40, 15: 42, 16: 19, 17: 21, 18: 22, 19: 24, 20: 25, 21: 26, 22: 27, 23: 28, 24: 30, 25: 31, 26: 34, 27: 35, 28: 14, 29: 15, 30: 17, 31: 39, 32: 38, 33: 41, 34: 43, 35: 20, 36: 23, 37: 29, 38: 33, 39: 32, 40: 1, 41: 0, 42: 2, 43: 3}


code to delete last three columns of atp2.gro and replace with last three columns of atp1.gro such that key (row) of atp1 matches with value (row) of atp2

In [5]:
!sed -i '$ d' atp2.gro

In [6]:
!sed -i 1,2d atp2.gro

In [7]:
!sed -i '$ d' atp1.gro

In [8]:
!sed -i 1,2d atp1.gro

In [9]:
import pandas as pd
atp2_df=pd.read_csv("atp2.gro",delimiter=r"\s+", header=None)
atp1_df=pd.read_csv("atp1.gro",delimiter=r"\s+", header=None)

In [10]:
for key in mapping:
    #print(key, "->", mapping[key])
    atp2_value=mapping[key]
    atp2_df.loc[atp2_value, 3]=atp1_df.loc[key, 3]
    atp2_df.loc[atp2_value, 4]=atp1_df.loc[key, 4]
    atp2_df.loc[atp2_value, 5]=atp1_df.loc[key, 5]
atp2_df.head()

0    1  2      3      4      5
0  1gtp  O1G  1 -1.290 -2.203 -1.865
1  1gtp   PG  2 -1.237 -2.271 -1.754
2  1gtp  O2G  3 -1.320 -2.380 -1.722
3  1gtp  O3G  4 -1.230 -2.179 -1.639
4  1gtp  O3B  5 -1.089 -2.324 -1.789

In [11]:
atp2_df.to_csv('output.txt', index=False, header=False, sep=' ')

In [12]:
!sed -i 's\1atp\1 atp\g' output.txt

In [13]:
!sed -i 's\1gtp\1 gtp\g' output.txt

In [14]:
def convert_to_gro(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        # Write the title for the .gro file
        outfile.write("generated\n")
        
        # Read all lines to count number of atoms
        lines = infile.readlines()
        number_of_atoms = len(lines)
        
        # Write the number of atoms in the .gro file
        outfile.write(f"{number_of_atoms}\n")
        
        for idx, line in enumerate(lines):
            columns = line.strip().split()
             
            # Adjust the columns as needed for your specific data
            res_num = int(columns[0])         # Residue number
            res_name = columns[1]             # Residue name
            atom_name = columns[2]            # Atom name
            atom_num = int(columns[3])        # Atom number
            x = float(columns[4])             # X coordinate
            y = float(columns[5])             # Y coordinate
            z = float(columns[6])             # Z coordinate
                        
            # Write the atom line in .gro format
            outfile.write(f"{res_num:5d}{res_name:<5}{atom_name:>5}{atom_num:5d}{x:8.3f}{y:8.3f}{z:8.3f}\n")
        
        # Write the box dimensions (example values, adjust as needed)
        outfile.write("   1.00000   1.00000   1.00000\n")

# Example usage
input_file = 'output.txt'
output_file = 'atp2.gro'
convert_to_gro(input_file, output_file)

In [15]:
!rm output.txt atp1.gro atp1.top

In [16]:
atp = BSS.IO.readMolecules(["atp2.top","atp2.gro"])[0]  
adp = BSS.IO.readMolecules(["inputs/ligands/gdp_maegher.prm7","inputs/ligands/gdp_maegher.rst7"])[0]   

In [17]:
mapping = BSS.Align.matchAtoms(atp, adp)
# Mapping is a dictionary mapping atom indices in ethane to those in methanol.
print(mapping)
inv_mapping = {v: k for k, v in mapping.items()}
adp_aligned = BSS.Align.rmsdAlign(adp, atp, inv_mapping)

{4: 0, 5: 1, 6: 3, 7: 2, 8: 4, 9: 5, 10: 7, 11: 6, 12: 8, 13: 9, 14: 10, 15: 11, 16: 12, 17: 13, 18: 14, 19: 15, 20: 16, 21: 17, 22: 18, 23: 19, 24: 20, 25: 21, 26: 22, 27: 23, 28: 24, 29: 25, 30: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 38, 38: 39, 39: 33, 40: 34, 41: 35, 42: 36, 43: 37}


We can now merge the two molecules. This will create a composite molecule containing all of the molecular properties at both end states. If the molecules are a different size, then the smaller will contain dummy atoms to represent the atoms that will appear during the perturbation. In this case, the merged methanol end state will contain two dummy atoms corresponding to the extra hydrogen atoms in the ethane molecule.

In [18]:
merged = BSS.Align.merge(atp, adp_aligned, mapping)
# solvated = BSS.Solvent.tip3p(molecule=merged, box=3*[50*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
# protocol = BSS.Protocol.FreeEnergyProduction(timestep=2*BSS.Units.Time.femtosecond, runtime=0.001*BSS.Units.Time.nanosecond, num_lam=3)
# fep_free = BSS.FreeEnergy.Relative(solvated, protocol, engine="GROMACS", work_dir="ligandleg_atp_adp_noMG", ignore_warnings=True, show_errors=True)
# fep_free.run()
# print("done")

In [19]:
protein = BSS.IO.readMolecules(["inputs/2qn6-missingloop-amber-withoutMG/topol.top","inputs/2qn6-missingloop-amber-withoutMG/step3_input.gro"])  #from minimised gromacs


In [20]:
complx = merged + protein

In [21]:
complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[70*BSS.Units.Length.angstrom], ion_conc=0.15, is_neutral=True)
protocol = BSS.Protocol.FreeEnergy(runtime=0.01*BSS.Units.Time.nanosecond, num_lam=3)
fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warnings=True, work_dir="atp_adp")
fep_bound.run()  
fep_bound.wait()
print("done")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 complex_sol = BSS.Solvent.tip3p(molecule=complx, box=3*[70*BSS.Units.Length.angstrom], i     │
│   2 protocol = BSS.Protocol.FreeEnergy(runtime=0.01*BSS.Units.Time.nanosecond, num_lam=3)        │
│ ❱ 3 fep_bound = BSS.FreeEnergy.Relative(complex_sol, protocol, engine="GROMACS", ignore_warn     │
│   4 fep_bound.run()                                                                              │
│   5 fep_bound.wait()                                                                             │
│   6 print("done")                                                                                │
│                                                                                                  │
│ /opt/miniforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/FreeEnergy/_relative.py:3 │
│ 33 in __init__                                                                                   │
│                                                                                                  │
│    330 │   │   self.difference = self._difference                                                │
│    331 │   │                                                                                     │
│    332 │   │   # Initialise the process runner.                                                  │
│ ❱  333 │   │   self._initialise_runner(self._system)                                             │
│    334 │                                                                                         │
│    335 │   def run(self, serial=True):                                                           │
│    336 │   │   """                                                                               │
│                                                                                                  │
│ /opt/miniforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/FreeEnergy/_relative.py:2 │
│ 053 in _initialise_runner                                                                        │
│                                                                                                  │
│   2050 │   │                                                                                     │
│   2051 │   │   # GROMACS.                                                                        │
│   2052 │   │   elif self._engine == "GROMACS":                                                   │
│ ❱ 2053 │   │   │   first_process = _Process.Gromacs(                                             │
│   2054 │   │   │   │   system,                                                                   │
│   2055 │   │   │   │   self._protocol,                                                           │
│   2056 │   │   │   │   work_dir=first_dir,                                                       │
│                                                                                                  │
│ /opt/miniforge/envs/openbiosim/lib/python3.9/site-packages/BioSimSpace/Process/_gromacs.py:225   │
│ in __init__                                                                                      │
│                                                                                                  │
│    222 │   │   │   │   │   )                                                                     │
│    223 │   │                                                                                     │
│    224 │   │   # Now set up the working directory for the process.                               │
│ ❱  225 │   │   self._setup()                                                                     │
│    226 │                                                   